<a href="https://colab.research.google.com/github/HenriqueCCdA/bootCampAluraDataScience/blob/master/modulo4/extra/ICU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1) Pre-analise

## 1.1) Importando libs

In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 100)

## 1.2) Funções

In [2]:
def pacientes_por_janela_ICU(dados):
  '''
  ------------------------------------------------------------------------------
  Função que conta se o paciente esta ou não na UTI para cada janela
  ------------------------------------------------------------------------------
  @param dados - dateFrame
  ------------------------------------------------------------------------------
  @return ICU_window_0_2  - numero de pacientes que foram para UTI janela 0-2
  @return ICU_window_2_4  - numero de pacientes que foram para UTI janela 2-4
  @return ICU_window_4_6  - numero de pacientes que foram para UTI janela 4-6
  @return ICU_window_6_12 - numero de pacientes que foram para UTI janela 6-12
  ------------------------------------------------------------------------------
  '''

  ICU_window_0_2 = dados.query("WINDOW == '0-2' and ICU==1")['PATIENT_VISIT_IDENTIFIER']
  ICU_window_2_4 = dados.query("WINDOW == '2-4' and ICU==1")['PATIENT_VISIT_IDENTIFIER']
  ICU_window_4_6 = dados.query("WINDOW == '4-6' and ICU==1")['PATIENT_VISIT_IDENTIFIER']
  ICU_window_6_12 = dados.query("WINDOW == '6-12' and ICU==1")['PATIENT_VISIT_IDENTIFIER']

  print('Numero de ICU igual a 1')
  print('Janela 0-2 :' ,len(ICU_window_0_2))
  print('Janela 2-4 :' ,len(ICU_window_2_4))
  print('Janela 4-6 :' ,len(ICU_window_4_6))
  print('Janela 6-12:',len(ICU_window_6_12))

  return ICU_window_0_2, ICU_window_2_4, ICU_window_4_6, ICU_window_6_12

In [3]:
def colunas_com_apenas_n_valores_unicos(dados, n=2, ci=13, cf=228):
  '''
  ------------------------------------------------------------------------------
  Mostra todas as colunas que so tem n dados unicos
  ------------------------------------------------------------------------------
  @param dados - dados
  @param n     - numero de dados unicos desejados
  @param ci    - coluna inicial
  @param cf    - coluna final
  ------------------------------------------------------------------------------
  '''

  print(f"Colunas com apenas {n} valores unicos:")
  for name in dados.columns[ci:cf]:
    if len( dados[name].unique()) < n + 1:
      print(f'{name:20} ->', dados[name].unique())

def retira_paciente_primeira_janela(matriz):
  '''
  ------------------------------------------------------------------------------
  Retira da base de dados os pacientes que foram para UTI na pimeira janela
  0-2
  ------------------------------------------------------------------------------
  @param matriz - dados brutos
  ------------------------------------------------------------------------------
  @return matriz sem o pacientes que que foram para UTI na pimeira janela
  ------------------------------------------------------------------------------
  '''

  window = matriz.query("WINDOW=='0-2' and ICU==1")['PATIENT_VISIT_IDENTIFIER']

  matriz = matriz.query("PATIENT_VISIT_IDENTIFIER not in @window")

  return matriz

def submatriz_preenchimento(submatriz):
  '''
  ------------------------------------------------------------------------------
  Preenche as variaveis continuas utilizando. Esta função trabalho com submatriz
  da matriz principal
  ------------------------------------------------------------------------------
  @param submatriz - submatriz agrupapor paciente e UCIs
  ------------------------------------------------------------------------------
  @return submatriz preenchida quando 
  ------------------------------------------------------------------------------
  '''
  
  submatriz_var_continuas = submatriz.iloc[:, 13:-2]
  submatriz_var_categorica_inicio = submatriz.iloc[:, :13]
  submatriz_var_categorica_final  = submatriz.iloc[:,-2:]

  # preenchendo os valores NaN 

  submatriz_var_continuas = submatriz_var_continuas.fillna(method='ffill')\
                                                   .fillna(method='bfill')

  submatriz_preenchida = pd.concat([submatriz_var_categorica_inicio, 
                                      submatriz_var_continuas,  
                                      submatriz_var_categorica_final],
                                      ignore_index=False, axis=1)
  
  return submatriz_preenchida


def preenchendo_var_continuas(matriz):

  '''
  ------------------------------------------------------------------------------
  Preenche as variaveis continuas utilizando os somente os dados para quando 
  ITU = 0. 
  ------------------------------------------------------------------------------
  @param matriz - dados brutos
  ------------------------------------------------------------------------------
  @return matriz sem o pacientes que que foram para UTI na pimeira janela
  ------------------------------------------------------------------------------
  '''

  matriz_preenchida = matriz.groupby(['PATIENT_VISIT_IDENTIFIER', 'ICU'],
                                     as_index=False).apply(submatriz_preenchimento)
  matriz_preenchida = matriz_preenchida.reset_index().drop(['level_0', 
                                                            'level_1'], axis=1)

  return matriz_preenchida


def uma_linha_por_paciente(submatriz):
  '''
  ------------------------------------------------------------------------------
  Reduz todas as janelas do paciente a apenas um linha. Os valores são pegos
  da janela 0-2
  ------------------------------------------------------------------------------
  @param submatriz - Submatriz agrupa por pacientes
  ------------------------------------------------------------------------------
  @return Retorna os valores da janela 0-2 com a informa se o paciente foi ou 
  não para a UTI
  ------------------------------------------------------------------------------
  '''

  if np.any(submatriz['ICU']):
    submatriz.iloc[:,-1] = 1

  return submatriz.iloc[0,:]

# 2) Explorando os dados

# 2.1) Importando os dados

In [4]:
path = "https://github.com/HenriqueCCdA/bootCampAluraDataScience/blob/master/modulo4/dados/Kaggle_Sirio_Libanes_ICU_Prediction.xlsx?raw=true"

dados = pd.read_excel(path)
dados.head(n=5)

,PATIENT_VISIT_IDENTIFIER,AGE_ABOVE65,AGE_PERCENTIL,GENDER,DISEASE GROUPING 1,DISEASE GROUPING 2,DISEASE GROUPING 3,DISEASE GROUPING 4,DISEASE GROUPING 5,DISEASE GROUPING 6,HTN,IMMUNOCOMPROMISED,OTHER,ALBUMIN_MEDIAN,ALBUMIN_MEAN,ALBUMIN_MIN,ALBUMIN_MAX,ALBUMIN_DIFF,BE_ARTERIAL_MEDIAN,BE_ARTERIAL_MEAN,BE_ARTERIAL_MIN,BE_ARTERIAL_MAX,BE_ARTERIAL_DIFF,BE_VENOUS_MEDIAN,BE_VENOUS_MEAN,BE_VENOUS_MIN,BE_VENOUS_MAX,BE_VENOUS_DIFF,BIC_ARTERIAL_MEDIAN,BIC_ARTERIAL_MEAN,BIC_ARTERIAL_MIN,BIC_ARTERIAL_MAX,BIC_ARTERIAL_DIFF,BIC_VENOUS_MEDIAN,BIC_VENOUS_MEAN,BIC_VENOUS_MIN,BIC_VENOUS_MAX,BIC_VENOUS_DIFF,BILLIRUBIN_MEDIAN,BILLIRUBIN_MEAN,...,DIMER_MAX,DIMER_DIFF,BLOODPRESSURE_DIASTOLIC_MEAN,BLOODPRESSURE_SISTOLIC_MEAN,HEART_RATE_MEAN,RESPIRATORY_RATE_MEAN,TEMPERATURE_MEAN,OXYGEN_SATURATION_MEAN,BLOODPRESSURE_DIASTOLIC_MEDIAN,BLOODPRESSURE_SISTOLIC_MEDIAN,HEART_RATE_MEDIAN,RESPIRATORY_RATE_MEDIAN,TEMPERATURE_MEDIAN,OXYGEN_SATURATION_MEDIAN,BLOODPRESSURE_DIASTOLIC_MIN,BLOODPRESSURE_SISTOLIC_MIN,HEART_RATE_MIN,RESPIRATORY_RATE_MIN,TEMPERATURE_MIN,OXYGEN_SATURATION_MIN,BLOODPRESSURE_DIASTOLIC_MAX,BLOODPRESSURE_SISTOLIC_MAX,HEART_RATE_MAX,RESPIRATORY_RATE_MAX,TEMPERATURE_MAX,OXYGEN_SATURATION_MAX,BLOODPRESSURE_DIASTOLIC_DIFF,BLOODPRESSURE_SISTOLIC_DIFF,HEART_RATE_DIFF,RESPIRATORY_RATE_DIFF,TEMPERATURE_DIFF,OXYGEN_SATURATION_DIFF,BLOODPRESSURE_DIASTOLIC_DIFF_REL,BLOODPRESSURE_SISTOLIC_DIFF_REL,HEART_RATE_DIFF_REL,RESPIRATORY_RATE_DIFF_REL,TEMPERATURE_DIFF_REL,OXYGEN_SATURATION_DIFF_REL,WINDOW,ICU
0,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.086420,-0.230769,-0.283019,-0.593220,-0.285714,0.736842,0.086420,-0.230769,-0.283019,-0.586207,-0.285714,0.736842,0.237113,0.0000,-0.162393,-0.500000,0.208791,0.898990,-0.247863,-0.459459,-0.432836,-0.636364,-0.420290,0.736842,-1.00000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0-2,0
1,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.333333,-0.230769,-0.132075,-0.593220,0.535714,0.578947,0.333333,-0.230769,-0.132075,-0.586207,0.535714,0.578947,0.443299,0.0000,-0.025641,-0.500000,0.714286,0.838384,-0.076923,-0.459459,-0.313433,-0.636364,0.246377,0.578947,-1.00000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,2-4,0
2,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.605263,0.605263,0.605263,0.605263,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.938950,-0.938950,...,-0.994912,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4-6,0
3,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-0.107143,0.736842,NaN,NaN,NaN,NaN,-0.107143,0.736842,NaN,NaN,NaN,NaN,0.318681,0.898990,NaN,NaN,NaN,NaN,-0.275362,0.736842,NaN,NaN,NaN,NaN,-1.000000,-1.000000,NaN,NaN,NaN,NaN,-1.000000,-1.000000,6-12,0
4,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.000000,0.000000,0.000000,0.000000,-1.0,-0.871658,-0.871658,-0.871658,-0.871658,-1.0,-0.863874,-0.863874,-0.863874,-0.863874,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.414634,-0.414634,-0.414634,-0.414634,-1.0,-0.979069,-0.979069,...,-0.996762,-1.0,-0.243021,-0.338537,-0.213031,-0.317859,0.033779,0.665932,-0.283951,-0.376923,-0.188679,-0.379310,0.035714,0.631579,-0.340206,-0.4875,-0.572650,-0.857143,0.098901,0.797980,-0.076923,0.286486,0.298507,0.272727,0.362319,0.947368,-0.33913,0.325153,0.114504,0.176471,-0.238095,-0.818182,-0.389967,0.407558,

In [5]:
nl, nc = dados.shape
print(f"Numero de linhas : {nl} ")
print(f"Numero de colunas: {nc} ")

Numero de linhas : 1925 
Numero de colunas: 231 


In [6]:
print(dados.dtypes)

PATIENT_VISIT_IDENTIFIER        int64
AGE_ABOVE65                     int64
AGE_PERCENTIL                  object
GENDER                          int64
DISEASE GROUPING 1            float64
                               ...   
RESPIRATORY_RATE_DIFF_REL     float64
TEMPERATURE_DIFF_REL          float64
OXYGEN_SATURATION_DIFF_REL    float64
WINDOW                         object
ICU                             int64
Length: 231, dtype: object


In [7]:
dados.describe()

,PATIENT_VISIT_IDENTIFIER,AGE_ABOVE65,GENDER,DISEASE GROUPING 1,DISEASE GROUPING 2,DISEASE GROUPING 3,DISEASE GROUPING 4,DISEASE GROUPING 5,DISEASE GROUPING 6,HTN,IMMUNOCOMPROMISED,OTHER,ALBUMIN_MEDIAN,ALBUMIN_MEAN,ALBUMIN_MIN,ALBUMIN_MAX,ALBUMIN_DIFF,BE_ARTERIAL_MEDIAN,BE_ARTERIAL_MEAN,BE_ARTERIAL_MIN,BE_ARTERIAL_MAX,BE_ARTERIAL_DIFF,BE_VENOUS_MEDIAN,BE_VENOUS_MEAN,BE_VENOUS_MIN,BE_VENOUS_MAX,BE_VENOUS_DIFF,BIC_ARTERIAL_MEDIAN,BIC_ARTERIAL_MEAN,BIC_ARTERIAL_MIN,BIC_ARTERIAL_MAX,BIC_ARTERIAL_DIFF,BIC_VENOUS_MEDIAN,BIC_VENOUS_MEAN,BIC_VENOUS_MIN,BIC_VENOUS_MAX,BIC_VENOUS_DIFF,BILLIRUBIN_MEDIAN,BILLIRUBIN_MEAN,BILLIRUBIN_MIN,...,DIMER_MIN,DIMER_MAX,DIMER_DIFF,BLOODPRESSURE_DIASTOLIC_MEAN,BLOODPRESSURE_SISTOLIC_MEAN,HEART_RATE_MEAN,RESPIRATORY_RATE_MEAN,TEMPERATURE_MEAN,OXYGEN_SATURATION_MEAN,BLOODPRESSURE_DIASTOLIC_MEDIAN,BLOODPRESSURE_SISTOLIC_MEDIAN,HEART_RATE_MEDIAN,RESPIRATORY_RATE_MEDIAN,TEMPERATURE_MEDIAN,OXYGEN_SATURATION_MEDIAN,BLOODPRESSURE_DIASTOLIC_MIN,BLOODPRESSURE_SISTOLIC_MIN,HEART_RATE_MIN,RESPIRATORY_RATE_MIN,TEMPERATURE_MIN,OXYGEN_SATURATION_MIN,BLOODPRESSURE_DIASTOLIC_MAX,BLOODPRESSURE_SISTOLIC_MAX,HEART_RATE_MAX,RESPIRATORY_RATE_MAX,TEMPERATURE_MAX,OXYGEN_SATURATION_MAX,BLOODPRESSURE_DIASTOLIC_DIFF,BLOODPRESSURE_SISTOLIC_DIFF,HEART_RATE_DIFF,RESPIRATORY_RATE_DIFF,TEMPERATURE_DIFF,OXYGEN_SATURATION_DIFF,BLOODPRESSURE_DIASTOLIC_DIFF_REL,BLOODPRESSURE_SISTOLIC_DIFF_REL,HEART_RATE_DIFF_REL,RESPIRATORY_RATE_DIFF_REL,TEMPERATURE_DIFF_REL,OXYGEN_SATURATION_DIFF_REL,ICU
count,1925.000000,1925.000000,1925.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,821.000000,821.000000,821.000000,821.000000,821.0,821.000000,821.000000,821.000000,821.000000,821.0,821.000000,821.000000,821.000000,821.000000,821.0,821.000000,821.000000,821.000000,821.000000,821.0,821.000000,821.000000,821.000000,821.000000,821.0,821.000000,821.000000,821.000000,...,821.000000,821.000000,821.0,1240.000000,1240.000000,1240.000000,1177.000000,1231.000000,1239.000000,1240.000000,1240.000000,1240.000000,1177.000000,1231.000000,1239.000000,1240.000000,1240.000000,1240.000000,1177.000000,1231.000000,1239.000000,1240.000000,1240.000000,1240.000000,1177.000000,1231.000000,1239.000000,1240.000000,1240.000000,1240.000000,1177.000000,1231.000000,1239.000000,1240.000000,1240.000000,1240.000000,1177.000000,1231.000000,1239.000000,1925.000000
mean,192.000000,0.467532,0.368831,0.108333,0.028125,0.097917,0.019792,0.128125,0.046875,0.213021,0.158333,0.809896,0.528527,0.528527,0.528527,0.528527,-1.0,-0.963433,-0.963433,-0.963433,-0.963433,-1.0,-0.931121,-0.931121,-0.931121,-0.931121,-1.0,-0.310924,-0.310924,-0.310924,-0.310924,-1.0,-0.311845,-0.311845,-0.311845,-0.311845,-1.0,-0.945928,-0.945928,-0.945928,...,-0.954177,-0.954177,-1.0,-0.093631,-0.332600,-0.264701,-0.438754,0.066893,0.743077,-0.097790,-0.338468,-0.268632,-0.435121,0.063798,0.748588,-0.040855,-0.207812,-0.264999,-0.483129,0.326823,0.817565,-0.235001,-0.399582,-0.282029,-0.316753,0.014964,0.818593,-0.752454,-0.728053,-0.754100,-0.703683,-0.770338,-0.887196,-0.786997,-0.715950,-0.817800,-0.719147,-0.771327,-0.886982,0.267532
std,111.168431,0.499074,0.482613,0.310882,0.165373,0.297279,0.139320,0.334316,0.211426,0.409549,0.365148,0.392485,0.224100,0.224100,0.224100,0.224100,0.0,0.160870,0.160870,0.160870,0.160870,0.0,0.169509,0.169509,0.169509,0.169509,0.0,0.100256,0.100256,0.100256,0.100256,0.0,0.118812,0.118812,0.118812,0.118812,0.0,0.076610,0.076610,0.076610,...,0.123582,0.123582,0.0,0.252064,0.274102,0.246760,0.217113,0.242858,0.132635,0.257733,0.277952,0.252709,0.225554,0.249208,0.125994,0.281304,0.277802,0.272725,0.278239,0.216198,0.283453,0.271123,0.287580,0.296247,0.402675,0.276163,0.141316,0.364001,0.408677,0.366349,0.482097,0.319001,0.296147,0.324754,0.419103,0.270217,0.446600,0.317694,0.296772,0.442787
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000

In [8]:
dados.iloc[:21,[0,-2,-1]]

,PATIENT_VISIT_IDENTIFIER,WINDOW,ICU
0,0,0-2,0
1,0,2-4,0
2,0,4-6,0
3,0,6-12,0
4,0,ABOVE_12,1
5,1,0-2,1
6,1,2-4,1
7,1,4-6,1
8,1,6-12,1
9,1,ABOVE_12,1


In [9]:
n_pacientes_inicial = len(dados['PATIENT_VISIT_IDENTIFIER'].unique())
print('Numero de pacientes', n_pacientes_inicial)

Numero de pacientes 385


In [10]:
dados['AGE_PERCENTIL'].unique()

array(['60th', '90th', '10th', '40th', '70th', '20th', '50th', '80th',
       '30th', 'Above 90th'], dtype=object)

In [11]:
names = ['HTN', 'IMMUNOCOMPROMISED', 'OTHER', 'GENDER', 'AGE_ABOVE65',
         'DISEASE GROUPING 1', 'DISEASE GROUPING 2', 'DISEASE GROUPING 3',
         'DISEASE GROUPING 4', 'DISEASE GROUPING 5', 'DISEASE GROUPING 6',
         'WINDOW', 'ICU', 'AGE_PERCENTIL']
for name in names:
  print(f'{name:20} ->', dados[name].unique())

HTN                  -> [ 0.  1. nan]
IMMUNOCOMPROMISED    -> [ 0.  1. nan]
OTHER                -> [ 1.  0. nan]
GENDER               -> [0 1]
AGE_ABOVE65          -> [1 0]
DISEASE GROUPING 1   -> [ 0.  1. nan]
DISEASE GROUPING 2   -> [ 0.  1. nan]
DISEASE GROUPING 3   -> [ 0.  1. nan]
DISEASE GROUPING 4   -> [ 0.  1. nan]
DISEASE GROUPING 5   -> [ 1.  0. nan]
DISEASE GROUPING 6   -> [ 1.  0. nan]
WINDOW               -> ['0-2' '2-4' '4-6' '6-12' 'ABOVE_12']
ICU                  -> [0 1]
AGE_PERCENTIL        -> ['60th' '90th' '10th' '40th' '70th' '20th' '50th' '80th' '30th'
 'Above 90th']


In [12]:
for name in dados.columns:
  if 'ALBUMIN' in name:
    print(name)

ALBUMIN_MEDIAN
ALBUMIN_MEAN
ALBUMIN_MIN
ALBUMIN_MAX
ALBUMIN_DIFF


In [13]:
colunas_com_apenas_n_valores_unicos(dados, 2)

Colunas com apenas 2 valores unicos:
ALBUMIN_DIFF         -> [nan -1.]
BE_ARTERIAL_DIFF     -> [nan -1.]
BE_VENOUS_DIFF       -> [nan -1.]
BIC_ARTERIAL_DIFF    -> [nan -1.]
BIC_VENOUS_DIFF      -> [nan -1.]
BILLIRUBIN_DIFF      -> [nan -1.]
BLAST_DIFF           -> [nan -1.]
CALCIUM_DIFF         -> [nan -1.]
CREATININ_DIFF       -> [nan -1.]
FFA_DIFF             -> [nan -1.]
GGT_DIFF             -> [nan -1.]
GLUCOSE_DIFF         -> [nan -1.]
HEMATOCRITE_DIFF     -> [nan -1.]
HEMOGLOBIN_DIFF      -> [nan -1.]
INR_DIFF             -> [nan -1.]
LACTATE_DIFF         -> [nan -1.]
LEUKOCYTES_DIFF      -> [nan -1.]
LINFOCITOS_DIFF      -> [nan -1.]
NEUTROPHILES_DIFF    -> [nan -1.]
P02_ARTERIAL_DIFF    -> [nan -1.]
P02_VENOUS_DIFF      -> [nan -1.]
PC02_ARTERIAL_DIFF   -> [nan -1.]
PC02_VENOUS_DIFF     -> [nan -1.]
PCR_DIFF             -> [nan -1.]
PH_ARTERIAL_DIFF     -> [nan -1.]
PH_VENOUS_DIFF       -> [nan -1.]
PLATELETS_DIFF       -> [nan -1.]
POTASSIUM_DIFF       -> [nan -1.]
SAT02_ARTER

## 2.2) Resumo:

>Geral:
* Número de linhas  : 1925 
* Número de colunas : 231
* Colunas 0, 1, 3 e 230 tipo int64
* Colunas 2 e 229 tipo string
* Colunas 4 a 228 tipo float64
* Número de pacientes: 385 (cada paciente tem mutiplicas entradas por janelas de tempo)

>Colunas categoricas 0 ou 1:
* HTN                  -> [ 0.  1. nan]
* IMMUNOCOMPROMISED    -> [ 0.  1. nan]
* OTHER                -> [ 1.  0. nan]
* GENDER               -> [0 1]
* AGE_ABOVE65          -> [1 0]
* DISEASE GROUPING 1   -> [ 0.  1. nan]
* DISEASE GROUPING 2   -> [ 0.  1. nan]
* DISEASE GROUPING 3   -> [ 0.  1. nan]
* DISEASE GROUPING 4   -> [ 0.  1. nan]
* DISEASE GROUPING 5   -> [ 1.  0. nan]
* DISEASE GROUPING 6   -> [ 1.  0. nan]

>Colunas categoricas Multi-Label:
* WINDOW               -> ['0-2' '2-4' '4-6' '6-12' 'ABOVE_12']
* AGE_PERCENTIL        -> ['60th' '90th' '10th' '40th' '70th' '20th' '50th' '80th' '30th'
 'Above 90th']

>Possivel divisão das idades:
* 10th - 0  a 10 anos
* 20th - 11 a 20 anos
* 30th - 21 a 30 anos
* 40th - 31 a 20 anos
* 50th - 41 a 50 anos
* 60th - 51 a 60 anos
* 70th - 71 a 80 anos
* 80th - 81 a 90 anos
* Above 90th - 91 a 1.. anos

>OBS:
* Hypertension (HTN) 
* Imunodepressivos (IMMUNOCOMPROMISED)  
* UTI (intensive care unit)

> Nas colunas 13 até 228 as variaveis são continuas
>
>Por exemplo:\
OXYGEN_SATURATION_MEDIAN - mediana \
OXYGEN_SATURATION_MEAN   - media\
OXYGEN_SATURATION_MIN    - minimo\
OXYGEN_SATURATION_MAX    - maximo\
OXYGEN_SATURATION_DIFF   - max - min         
OXYGEN_SATURATION_DIFF_REL  - diff/median - (max - min)/median  (*nem todos tem)

>As colunas parecem ter apenas o valor -1\
ALBUMIN_DIFF        \
DE_ARTERIAL_DIFF    \
BE_VENOUS_DIFF      \
BIC_ARTERIAL_DIFF   \
BIC_VENOUS_DIFF     \
BILLIRUBIN_DIFF     \
BLAST_DIFF          \
CALCIUM_DIFF        \
CREATININ_DIFF      \
FFA_DIFF            \
GGT_DIFF            \
GLUCOSE_DIFF        \
HEMATOCRITE_DIFF    \
HEMOGLOBIN_DIFF     \
INR_DIFF            \
LACTATE_DIFF        \
LEUKOCYTES_DIFF     \
LINFOCITOS_DIFF     \
NEUTROPHILES_DIFF   \
P02_ARTERIAL_DIFF   \
P02_VENOUS_DIFF     \
PC02_ARTERIAL_DIFF  \
PC02_VENOUS_DIFF    \
PCR_DIFF            \
PH_ARTERIAL_DIFF    \
PH_VENOUS_DIFF      \
PLATELETS_DIFF      \
POTASSIUM_DIFF      \
SAT02_ARTERIAL_DIFF \
SAT02_VENOUS_DIFF   \
SODIUM_DIFF         \
TGO_DIFF            \
TGP_DIFF            \
TTPA_DIFF           \
UREA_DIFF           \
DIMER_DIFF



# 3) Tratamento dos dados

In [14]:
dados.iloc[:21,[0,-2,-1]]

,PATIENT_VISIT_IDENTIFIER,WINDOW,ICU
0,0,0-2,0
1,0,2-4,0
2,0,4-6,0
3,0,6-12,0
4,0,ABOVE_12,1
5,1,0-2,1
6,1,2-4,1
7,1,4-6,1
8,1,6-12,1
9,1,ABOVE_12,1


In [15]:
ICU_window_0_2, _, _, _ = pacientes_por_janela_ICU(dados)

Numero de ICU igual a 1
Janela 0-2 : 32
Janela 2-4 : 59
Janela 4-6 : 99
Janela 6-12: 130


In [16]:
ICU_window_0_2 # pacientes que precisam ser retirados

5         1
85       17
135      27
205      41
235      47
240      48
270      54
310      62
425      85
450      90
520     104
530     106
545     109
585     117
720     144
880     176
1010    202
1030    206
1130    226
1165    233
1195    239
1305    261
1350    270
1410    282
1430    286
1530    306
1575    315
1645    329
1795    359
1810    362
1830    366
1895    379
Name: PATIENT_VISIT_IDENTIFIER, dtype: int64

## 3.2) Manipulando os dados

* Retirando os pacientes que foram para a UTI na janela 0-2
* Preenchimento dos valores NaN nas variaveis continuas
* retirandos os valores NaN que restaram

In [17]:
dados_tratados = dados.copy()
# Retirando os pacientes que foram para a UTI na janela 0-2
dados_tratados = retira_paciente_primeira_janela(dados_tratados)
# Preenchimento dos valores NaN nas variaveis continuas
dados_tratados = preenchendo_var_continuas(dados_tratados)
# retira todas alinhas que ainda tem valores NaN
dados_tratados = dados_tratados.dropna()

In [18]:
dados_tratados.head(n=15)

,PATIENT_VISIT_IDENTIFIER,AGE_ABOVE65,AGE_PERCENTIL,GENDER,DISEASE GROUPING 1,DISEASE GROUPING 2,DISEASE GROUPING 3,DISEASE GROUPING 4,DISEASE GROUPING 5,DISEASE GROUPING 6,HTN,IMMUNOCOMPROMISED,OTHER,ALBUMIN_MEDIAN,ALBUMIN_MEAN,ALBUMIN_MIN,ALBUMIN_MAX,ALBUMIN_DIFF,BE_ARTERIAL_MEDIAN,BE_ARTERIAL_MEAN,BE_ARTERIAL_MIN,BE_ARTERIAL_MAX,BE_ARTERIAL_DIFF,BE_VENOUS_MEDIAN,BE_VENOUS_MEAN,BE_VENOUS_MIN,BE_VENOUS_MAX,BE_VENOUS_DIFF,BIC_ARTERIAL_MEDIAN,BIC_ARTERIAL_MEAN,BIC_ARTERIAL_MIN,BIC_ARTERIAL_MAX,BIC_ARTERIAL_DIFF,BIC_VENOUS_MEDIAN,BIC_VENOUS_MEAN,BIC_VENOUS_MIN,BIC_VENOUS_MAX,BIC_VENOUS_DIFF,BILLIRUBIN_MEDIAN,BILLIRUBIN_MEAN,...,DIMER_MAX,DIMER_DIFF,BLOODPRESSURE_DIASTOLIC_MEAN,BLOODPRESSURE_SISTOLIC_MEAN,HEART_RATE_MEAN,RESPIRATORY_RATE_MEAN,TEMPERATURE_MEAN,OXYGEN_SATURATION_MEAN,BLOODPRESSURE_DIASTOLIC_MEDIAN,BLOODPRESSURE_SISTOLIC_MEDIAN,HEART_RATE_MEDIAN,RESPIRATORY_RATE_MEDIAN,TEMPERATURE_MEDIAN,OXYGEN_SATURATION_MEDIAN,BLOODPRESSURE_DIASTOLIC_MIN,BLOODPRESSURE_SISTOLIC_MIN,HEART_RATE_MIN,RESPIRATORY_RATE_MIN,TEMPERATURE_MIN,OXYGEN_SATURATION_MIN,BLOODPRESSURE_DIASTOLIC_MAX,BLOODPRESSURE_SISTOLIC_MAX,HEART_RATE_MAX,RESPIRATORY_RATE_MAX,TEMPERATURE_MAX,OXYGEN_SATURATION_MAX,BLOODPRESSURE_DIASTOLIC_DIFF,BLOODPRESSURE_SISTOLIC_DIFF,HEART_RATE_DIFF,RESPIRATORY_RATE_DIFF,TEMPERATURE_DIFF,OXYGEN_SATURATION_DIFF,BLOODPRESSURE_DIASTOLIC_DIFF_REL,BLOODPRESSURE_SISTOLIC_DIFF_REL,HEART_RATE_DIFF_REL,RESPIRATORY_RATE_DIFF_REL,TEMPERATURE_DIFF_REL,OXYGEN_SATURATION_DIFF_REL,WINDOW,ICU
0,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.605263,0.605263,0.605263,0.605263,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.938950,-0.938950,...,-0.994912,-1.0,0.086420,-0.230769,-0.283019,-0.593220,-0.285714,0.736842,0.086420,-0.230769,-0.283019,-0.586207,-0.285714,0.736842,0.237113,0.0000,-0.162393,-0.500000,0.208791,0.898990,-0.247863,-0.459459,-0.432836,-0.636364,-0.420290,0.736842,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0-2,0
1,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.605263,0.605263,0.605263,0.605263,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.938950,-0.938950,...,-0.994912,-1.0,0.333333,-0.230769,-0.132075,-0.593220,0.535714,0.578947,0.333333,-0.230769,-0.132075,-0.586207,0.535714,0.578947,0.443299,0.0000,-0.025641,-0.500000,0.714286,0.838384,-0.076923,-0.459459,-0.313433,-0.636364,0.246377,0.578947,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,2-4,0
2,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.605263,0.605263,0.605263,0.605263,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.938950,-0.938950,...,-0.994912,-1.0,0.333333,-0.230769,-0.132075,-0.593220,0.535714,0.578947,0.333333,-0.230769,-0.132075,-0.586207,0.535714,0.578947,0.443299,0.0000,-0.025641,-0.500000,0.714286,0.838384,-0.076923,-0.459459,-0.313433,-0.636364,0.246377,0.578947,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,4-6,0
3,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.605263,0.605263,0.605263,0.605263,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.938950,-0.938950,...,-0.994912,-1.0,0.333333,-0.230769,-0.132075,-0.593220,-0.107143,0.736842,0.333333,-0.230769,-0.132075,-0.586207,-0.107143,0.736842,0.443299,0.0000,-0.025641,-0.500000,0.318681,0.898990,-0.076923,-0.459459,-0.313433,-0.63

##3.2.1) Checando se os pacientes foram corretamente retirados

In [19]:
_, _, _, _, = pacientes_por_janela_ICU(dados_tratados) # verificando se tem algum paciente ICU no intervalo 0-2

Numero de ICU igual a 1
Janela 0-2 : 0
Janela 2-4 : 27
Janela 4-6 : 67
Janela 6-12: 98


##3.2.2)  Checando se anda existem NaN

In [20]:
for col in dados_tratados:
  if dados_tratados[col].isna().sum():
    print("Valores NaN na coluna", col, dados_tratados[col].isna().sum())

In [21]:
nl, nc = dados_tratados.shape
print(f"Numero de linhas : {nl} ")
print(f"Numero de colunas: {nc} ")

Numero de linhas : 1645 
Numero de colunas: 231 


In [22]:
dados_tratados['AGE_PERCENTIL'].unique()

array(['60th', '10th', '40th', '70th', '20th', '50th', '80th', '30th',
       '90th', 'Above 90th'], dtype=object)

## 3.3) Modificando a coluna AGE_PERCENTIL

AGE_PERCENTIL| valor proposto |
:-----------:|:--------------:|
   10th      |    0  |
   20th      |    1  |          
   30th      |    2  |     
   40th      |    3  |     
   50th      |    4  | 
   60th      |    5  |       
   70th      |    6  | 
   80th      |    7  |
   90th      |    8  |
   Above 90th|    9  |          


In [33]:
valores = np.array([0,1,2,3,4,5,6,7,8,9])

media = np.mean(valores)
z = valores - media
max   = np.abs(np.max(z))
z = z / max
z

array([-1.        , -0.77777778, -0.55555556, -0.33333333, -0.11111111,
        0.11111111,  0.33333333,  0.55555556,  0.77777778,  1.        ])

In [34]:
map = { '10th'      : z[0],
        '20th'      : z[1],
        '30th'      : z[2],
        '40th'      : z[3],
        '50th'      : z[4],
        '60th'      : z[5],
        '70th'      : z[6],
        '80th'      : z[7],
        '90th'      : z[8],
        'Above 90th': z[9],
} 

In [35]:
dados_tratados['AGE_PERCENTIL'] = dados_tratados['AGE_PERCENTIL'].map(map)
dados_tratados['AGE_PERCENTIL'].dtype # checando o tipo

dtype('float64')

In [36]:
dados_tratados['AGE_PERCENTIL'].unique() # verifica se deu certo a substituicao

array([ 0.11111111, -1.        , -0.33333333,  0.33333333, -0.77777778,
       -0.11111111,  0.55555556, -0.55555556,  0.77777778,  1.        ])

In [37]:
dados_tratados.head(n=15)

,PATIENT_VISIT_IDENTIFIER,AGE_ABOVE65,AGE_PERCENTIL,GENDER,DISEASE GROUPING 1,DISEASE GROUPING 2,DISEASE GROUPING 3,DISEASE GROUPING 4,DISEASE GROUPING 5,DISEASE GROUPING 6,HTN,IMMUNOCOMPROMISED,OTHER,ALBUMIN_MEDIAN,ALBUMIN_MEAN,ALBUMIN_MIN,ALBUMIN_MAX,ALBUMIN_DIFF,BE_ARTERIAL_MEDIAN,BE_ARTERIAL_MEAN,BE_ARTERIAL_MIN,BE_ARTERIAL_MAX,BE_ARTERIAL_DIFF,BE_VENOUS_MEDIAN,BE_VENOUS_MEAN,BE_VENOUS_MIN,BE_VENOUS_MAX,BE_VENOUS_DIFF,BIC_ARTERIAL_MEDIAN,BIC_ARTERIAL_MEAN,BIC_ARTERIAL_MIN,BIC_ARTERIAL_MAX,BIC_ARTERIAL_DIFF,BIC_VENOUS_MEDIAN,BIC_VENOUS_MEAN,BIC_VENOUS_MIN,BIC_VENOUS_MAX,BIC_VENOUS_DIFF,BILLIRUBIN_MEDIAN,BILLIRUBIN_MEAN,...,DIMER_MAX,DIMER_DIFF,BLOODPRESSURE_DIASTOLIC_MEAN,BLOODPRESSURE_SISTOLIC_MEAN,HEART_RATE_MEAN,RESPIRATORY_RATE_MEAN,TEMPERATURE_MEAN,OXYGEN_SATURATION_MEAN,BLOODPRESSURE_DIASTOLIC_MEDIAN,BLOODPRESSURE_SISTOLIC_MEDIAN,HEART_RATE_MEDIAN,RESPIRATORY_RATE_MEDIAN,TEMPERATURE_MEDIAN,OXYGEN_SATURATION_MEDIAN,BLOODPRESSURE_DIASTOLIC_MIN,BLOODPRESSURE_SISTOLIC_MIN,HEART_RATE_MIN,RESPIRATORY_RATE_MIN,TEMPERATURE_MIN,OXYGEN_SATURATION_MIN,BLOODPRESSURE_DIASTOLIC_MAX,BLOODPRESSURE_SISTOLIC_MAX,HEART_RATE_MAX,RESPIRATORY_RATE_MAX,TEMPERATURE_MAX,OXYGEN_SATURATION_MAX,BLOODPRESSURE_DIASTOLIC_DIFF,BLOODPRESSURE_SISTOLIC_DIFF,HEART_RATE_DIFF,RESPIRATORY_RATE_DIFF,TEMPERATURE_DIFF,OXYGEN_SATURATION_DIFF,BLOODPRESSURE_DIASTOLIC_DIFF_REL,BLOODPRESSURE_SISTOLIC_DIFF_REL,HEART_RATE_DIFF_REL,RESPIRATORY_RATE_DIFF_REL,TEMPERATURE_DIFF_REL,OXYGEN_SATURATION_DIFF_REL,WINDOW,ICU
0,0,1,0.111111,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.605263,0.605263,0.605263,0.605263,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.938950,-0.938950,...,-0.994912,-1.0,0.086420,-0.230769,-0.283019,-0.593220,-0.285714,0.736842,0.086420,-0.230769,-0.283019,-0.586207,-0.285714,0.736842,0.237113,0.0000,-0.162393,-0.500000,0.208791,0.898990,-0.247863,-0.459459,-0.432836,-0.636364,-0.420290,0.736842,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0-2,0
1,0,1,0.111111,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.605263,0.605263,0.605263,0.605263,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.938950,-0.938950,...,-0.994912,-1.0,0.333333,-0.230769,-0.132075,-0.593220,0.535714,0.578947,0.333333,-0.230769,-0.132075,-0.586207,0.535714,0.578947,0.443299,0.0000,-0.025641,-0.500000,0.714286,0.838384,-0.076923,-0.459459,-0.313433,-0.636364,0.246377,0.578947,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,2-4,0
2,0,1,0.111111,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.605263,0.605263,0.605263,0.605263,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.938950,-0.938950,...,-0.994912,-1.0,0.333333,-0.230769,-0.132075,-0.593220,0.535714,0.578947,0.333333,-0.230769,-0.132075,-0.586207,0.535714,0.578947,0.443299,0.0000,-0.025641,-0.500000,0.714286,0.838384,-0.076923,-0.459459,-0.313433,-0.636364,0.246377,0.578947,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,4-6,0
3,0,1,0.111111,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.605263,0.605263,0.605263,0.605263,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.938950,-0.938950,...,-0.994912,-1.0,0.333333,-0.230769,-0.132075,-0.593220,-0.107143,0.736842,0.333333,-0.230769,-0.132075,-0.586207,-0.107143,0.736842,0.443299,0.0000,-0.025641,-0.500000,0.318681,0.898990,-0.076923,-0.459459

In [38]:
dados_tratados_por_paciente = dados_tratados.groupby('PATIENT_VISIT_IDENTIFIER', as_index=False).apply(uma_linha_por_paciente)

In [39]:
dados_tratados_por_paciente.head()

,PATIENT_VISIT_IDENTIFIER,AGE_ABOVE65,AGE_PERCENTIL,GENDER,DISEASE GROUPING 1,DISEASE GROUPING 2,DISEASE GROUPING 3,DISEASE GROUPING 4,DISEASE GROUPING 5,DISEASE GROUPING 6,HTN,IMMUNOCOMPROMISED,OTHER,ALBUMIN_MEDIAN,ALBUMIN_MEAN,ALBUMIN_MIN,ALBUMIN_MAX,ALBUMIN_DIFF,BE_ARTERIAL_MEDIAN,BE_ARTERIAL_MEAN,BE_ARTERIAL_MIN,BE_ARTERIAL_MAX,BE_ARTERIAL_DIFF,BE_VENOUS_MEDIAN,BE_VENOUS_MEAN,BE_VENOUS_MIN,BE_VENOUS_MAX,BE_VENOUS_DIFF,BIC_ARTERIAL_MEDIAN,BIC_ARTERIAL_MEAN,BIC_ARTERIAL_MIN,BIC_ARTERIAL_MAX,BIC_ARTERIAL_DIFF,BIC_VENOUS_MEDIAN,BIC_VENOUS_MEAN,BIC_VENOUS_MIN,BIC_VENOUS_MAX,BIC_VENOUS_DIFF,BILLIRUBIN_MEDIAN,BILLIRUBIN_MEAN,...,DIMER_MAX,DIMER_DIFF,BLOODPRESSURE_DIASTOLIC_MEAN,BLOODPRESSURE_SISTOLIC_MEAN,HEART_RATE_MEAN,RESPIRATORY_RATE_MEAN,TEMPERATURE_MEAN,OXYGEN_SATURATION_MEAN,BLOODPRESSURE_DIASTOLIC_MEDIAN,BLOODPRESSURE_SISTOLIC_MEDIAN,HEART_RATE_MEDIAN,RESPIRATORY_RATE_MEDIAN,TEMPERATURE_MEDIAN,OXYGEN_SATURATION_MEDIAN,BLOODPRESSURE_DIASTOLIC_MIN,BLOODPRESSURE_SISTOLIC_MIN,HEART_RATE_MIN,RESPIRATORY_RATE_MIN,TEMPERATURE_MIN,OXYGEN_SATURATION_MIN,BLOODPRESSURE_DIASTOLIC_MAX,BLOODPRESSURE_SISTOLIC_MAX,HEART_RATE_MAX,RESPIRATORY_RATE_MAX,TEMPERATURE_MAX,OXYGEN_SATURATION_MAX,BLOODPRESSURE_DIASTOLIC_DIFF,BLOODPRESSURE_SISTOLIC_DIFF,HEART_RATE_DIFF,RESPIRATORY_RATE_DIFF,TEMPERATURE_DIFF,OXYGEN_SATURATION_DIFF,BLOODPRESSURE_DIASTOLIC_DIFF_REL,BLOODPRESSURE_SISTOLIC_DIFF_REL,HEART_RATE_DIFF_REL,RESPIRATORY_RATE_DIFF_REL,TEMPERATURE_DIFF_REL,OXYGEN_SATURATION_DIFF_REL,WINDOW,ICU
0,0,1,0.111111,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.605263,0.605263,0.605263,0.605263,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.938950,-0.938950,...,-0.994912,-1.0,0.086420,-0.230769,-0.283019,-0.593220,-0.285714,0.736842,0.086420,-0.230769,-0.283019,-0.586207,-0.285714,0.736842,0.237113,0.0000,-0.162393,-0.500000,0.208791,0.898990,-0.247863,-0.459459,-0.432836,-0.636364,-0.420290,0.736842,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0-2,1
1,2,0,-1.000000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.605263,0.605263,0.605263,0.605263,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.938950,-0.938950,...,-0.978029,-1.0,-0.489712,-0.685470,-0.048218,-0.645951,0.357143,0.935673,-0.506173,-0.815385,-0.056604,-0.517241,0.357143,0.947368,-0.525773,-0.5125,-0.111111,-0.714286,0.604396,0.959596,-0.435897,-0.491892,0.000000,-0.575758,0.101449,1.000000,-0.547826,-0.533742,-0.603053,-0.764706,-1.000000,-0.959596,-0.515528,-0.351328,-0.747001,-0.756272,-1.000000,-0.961262,0-2,1
2,3,0,-0.333333,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,-0.263158,-0.263158,-0.263158,-0.263158,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.972789,-0.972789,...,-0.978029,-1.0,0.012346,-0.369231,-0.528302,-0.457627,-0.285714,0.684211,0.012346,-0.369231,-0.528302,-0.448276,-0.285714,0.684211,0.175258,-0.1125,-0.384615,-0.357143,0.208791,0.878788,-0.299145,-0.556757,-0.626866,-0.515152,-0.420290,0.684211,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0-2,0
3,4,0,-1.000000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.605263,0.605263,0.605263,0.605263,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.935113,-0.935113,...,-1.000000,-1.0,0.333333,-0.153846,0.160377,-0.593220,0.285714,0.868421,0.333333,-0.153846,0.160377,-0.586207,0.285714,0.868421,0.443299,0.0000,0.196581,-0.571429,0.538462,0.939394,-0.076923,-0.351351,-0.044776,-0.575758,0.072464,0.894737,-1.000000,-0.877301,-0.923664,-0.882353,-0.952381,-0.979798,-1.000000,-0.883669,-0.956805,-0.870968,-0.953536,-0

In [40]:
n_pacientes, _ = dados_tratados_por_paciente.shape
print(f"Numero de pacientes inicial : {n_pacientes_inicial}")
print(f"Numero de pacientes validos : {n_pacientes} ")

Numero de pacientes inicial : 385
Numero de pacientes validos : 351 


In [41]:
colunas_com_apenas_n_valores_unicos(dados_tratados_por_paciente, 2)

Colunas com apenas 2 valores unicos:
ALBUMIN_DIFF         -> [-1.]
BE_ARTERIAL_DIFF     -> [-1.]
BE_VENOUS_DIFF       -> [-1.]
BIC_ARTERIAL_DIFF    -> [-1.]
BIC_VENOUS_DIFF      -> [-1.]
BILLIRUBIN_DIFF      -> [-1.]
BLAST_DIFF           -> [-1.]
CALCIUM_DIFF         -> [-1.]
CREATININ_DIFF       -> [-1.]
FFA_DIFF             -> [-1.]
GGT_DIFF             -> [-1.]
GLUCOSE_DIFF         -> [-1.]
HEMATOCRITE_DIFF     -> [-1.]
HEMOGLOBIN_DIFF      -> [-1.]
INR_DIFF             -> [-1.]
LACTATE_DIFF         -> [-1.]
LEUKOCYTES_DIFF      -> [-1.]
LINFOCITOS_DIFF      -> [-1.]
NEUTROPHILES_DIFF    -> [-1.]
P02_ARTERIAL_DIFF    -> [-1.]
P02_VENOUS_DIFF      -> [-1.]
PC02_ARTERIAL_DIFF   -> [-1.]
PC02_VENOUS_DIFF     -> [-1.]
PCR_DIFF             -> [-1.]
PH_ARTERIAL_DIFF     -> [-1.]
PH_VENOUS_DIFF       -> [-1.]
PLATELETS_DIFF       -> [-1.]
POTASSIUM_DIFF       -> [-1.]
SAT02_ARTERIAL_DIFF  -> [-1.]
SAT02_VENOUS_DIFF    -> [-1.]
SODIUM_DIFF          -> [-1.]
TGO_DIFF             -> [-1.]
TGP

# 4) Salvando os dados tratados 

In [43]:
dados_tratados_por_paciente.to_csv('dados_tratados_por_paciente.csv', index_label=False)